# read_tmdb_data.ipynb

This notebook reads JSON data from TMDB's API and store it as CSV files:
- **danish_movies.csv**: Data on all Danish-language movies ever made up to and including 2024-12-31.
- **danish_actors.csv**: Data on all actors that appear in these movies (also foreign actors).

In [1]:
import os
import requests
from dotenv import load_dotenv
from time import sleep
import pandas as pd

load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}

First we request movie_ids for all movies that have danish as original_language

In [2]:
base_url = (
    "https://api.themoviedb.org/3/discover/movie"
    "?include_adult=false"
    "&include_video=false"
    "&language=en-US"
    "&sort_by=primary_release_date.asc"
    "&with_original_language=da"
    "&primary_release_date.gte=1800-01-01"
    "&primary_release_date.lte=2024-12-31"
)

movie_ids = []

# Loop through all pages (adjust max page if needed)
for page in range(1, 501):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Status code: {response.status_code} Text: {response.text}")
    
    # Extract movie_results from the response
    movie_results = response.json().get("results") 
    
    # Stop if we have reached the last page
    if not movie_results:
        break

    # Extract movie_ids from the movie_results
    movie_ids.extend([movie["id"] for movie in movie_results])
    
    sleep(0.02)  # Just to be sure we don't request too many requests

print(f"Number of movie ids found: {len(movie_ids)}")

Number of movie ids found: 5134


Then we request movie data for all these movie_ids, and store that in a dataframe and a CSV file

In [3]:
all_movies = []

# Loop through all movie_ids and request movie data
for movie_id in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?language=en-US&append_to_response=credits"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Status code: {response.status_code} Text: {response.text}")
    
    # Extract the movie data from the response
    movie = response.json()

    # Simplify dictionaries to list of ids
    movie["genre_ids"] = [genre["id"] for genre in movie["genres"]]
    movie["spoken_languages"] = [language["iso_639_1"] for language in movie["spoken_languages"]]
    movie["production_company_ids"] = [company["id"] for company in movie["production_companies"]]
    movie["production_countries"] = [country["iso_3166_1"] for country in movie["production_countries"]]
    movie["collection_id"] = movie.get("belongs_to_collection").get("id") if movie["belongs_to_collection"] else None
    movie["cast_person_ids"] = [cast_member["id"] for cast_member in movie["credits"]["cast"]]
    movie["cast_credit_ids"] = [cast_member["credit_id"] for cast_member in movie["credits"]["cast"]]
    movie["crew_person_ids"] = [crew_member["id"] for crew_member in movie["credits"]["crew"]]
    movie["crew_credit_ids"] = [crew_member["credit_id"] for crew_member in movie["credits"]["crew"]]
    del movie['genres']
    del movie['production_companies']
    del movie["belongs_to_collection"]   
    del movie["credits"]
    
    all_movies.append(movie)
    sleep(0.02)  # Just to be sure we don't request too many requests

df_movies = pd.DataFrame(all_movies)
print(f"Total movies collected: {len(df_movies)}")

df_movies.to_csv("../movie_data/danish_movies.csv", index=False)

Total movies collected: 5134


In [4]:
df_movies

,adult,backdrop_path,budget,homepage,id,imdb_id,origin_country,original_language,original_title,overview,...,video,vote_average,vote_count,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
0,False,None,0,,195139,tt0290709,[DK],da,Kørsel med Grønlandske hunde,"Johan Carl Joensen, a Danish colonial manager ...",...,False,5.3,15,[99],[41774],NaN,[2452695],[5dc5d410470ead001391303b],"[1171313, 1171313]","[52fe4d069251416c91108121, 5dc5d3ef470ead00159..."
1,False,None,0,,232636,tt0348157,[DK],da,Svanerne i Sortedamssøen,Swans getting handled as people watch.,...,False,3.0,2,[99],[41774],NaN,[],[],[1171313],[52fe4e07c3a36847f827a04d]
2,False,None,0,,195364,tt0346587,[DK],da,Bech Olsens og Poul Pons' brydekamp,Early wrestling footage.,...,False,4.0,5,[99],[41774],NaN,"[4658231, 4658234]","[661fe77220af77017d3f88a8, 661fe78a6d9fe8017d6...",[1171313],[52fe4d0d9251416c91108ee5]
3,False,None,0,,232637,tt1469330,[DK],da,Badescener fra Skovshoved,Fun at the beach.,...,False,4.5,4,[99],[41774],NaN,[],[],[1171313],[52fe4e07c3a36847f827a057]
4,False,None,0,,195305,tt0347357,[DK],da,De Kongelige skal fotograferes,"Filmed while the Christian IX, King of Denmark...",...,False,4.8,4,[99],[],NaN,[],[],[1171313],[52fe4d0b9251416c91108c3d]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5129,False,/MSP35R6y0ypmf3zU1kyUw1P4It.jpg,0,,1407803,None,[DK],da,Verdensmænd - Bobos surprise,,...,False,10.0,1,[35],[758],NaN,"[1828049, 234058, 1355004, 88356, 2208065]","[676d44614ca2fbbfca614283, 676d44784ca2fbbfca6...",[],[]
5130,False,/2iVVzCEnX4QMEeyd3e52juYcsH8.jpg,0,,1410638,None,[DK],da,Gud bevare Danmark,,...,False,0.0,0,[],[],NaN,[],[],[],[]
5131,False,/ej5uQVMJTzq8nZBCbf6pUMNF6jl.jpg,0,,1409581,None,[DK],da,Frank Hvam - Nobody,Frank Hvam has lived in New Zealand with his f...,...,False,6.0,1,[35],[758],NaN,[89973],[67724668d8dc22e6c6927586],"[89973, 5520795, 563909, 5227452, 4263643, 516...","[68650819af3c4b2566414b34, 686508769febd451ea8..."
5132,False,None,0,,1412034,None,[DK],da,"Dan Andersen - Nedsat Hørelse, Nedsat Sædkvali...",,...,False,0.0,0,[35],[],NaN,[149860],[677903662b097b15a274ab1e],[],[]


Then we run through the movie dataframe and find all unique actor_ids.

And then we request data for all these actor_ids, and store that in a dataframe and a CSV file

In [5]:
# Step 1: Explode the cast_person_ids list so each actor appears in a separate row
df_exploded = df_movies[['id', 'cast_person_ids']].explode('cast_person_ids')

In [6]:
# Step 2: Rename for clarity
df_exploded = df_exploded.rename(columns={'id': 'movie_ids', 'cast_person_ids': 'actor_id'})

In [7]:
# Step 3: Group by actor_id and aggregate the movie_ids into a set
danish_actors = df_exploded.groupby('actor_id')['movie_ids'].agg(set).reset_index()

In [8]:
all_persons = []

for actor_id in danish_actors['actor_id']:
    url = f"https://api.themoviedb.org/3/person/{actor_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Status code: {response.status_code} Text: {response.text}")
    
    # Extract the person data from the response
    person = response.json()
    
    all_persons.append(person)
    sleep(0.02)  # Just to be sure we don't request too many requests
    
    
df_persons = pd.DataFrame(all_persons)


In [9]:
df_persons.rename(columns={'id': 'actor_id'}, inplace=True)
danish_actors = pd.merge(danish_actors, df_persons, on='actor_id')
danish_actors

,actor_id,movie_ids,adult,also_known_as,biography,birthday,deathday,gender,homepage,imdb_id,known_for_department,name,place_of_birth,popularity,profile_path
0,42,"{658817, 456325, 29445, 1354636, 752908, 14460...",False,"[Ларс фон Триер, 拉斯·馮·提爾, ラース・フォン・トリアー, لارس ف...",Lars von Trier (born Lars Trier; 30 April 1956...,1956-04-30,None,2,None,nm0001885,Directing,Lars von Trier,"Kongens Lyngby, Danmark",0.9345,/a1zVySI0Yqqf3ORegQ1icMfvhAa.jpg
1,53,{15843},False,"[Rolf Peter Ingvar Storm, پیتر استورماره, پِتِ...","Peter Stormare (August 27, 1953) was born in A...",1953-08-27,None,2,None,nm0001780,Acting,Peter Stormare,"Arbrå, Gävleborgs län, Sweden",4.6882,/1rtpuUqBV29jDc1huUhtjGDbEwn.jpg
2,169,{8883},False,[],Hanns Zischler (born 18 June 1947) is a German...,1947-06-18,None,2,None,nm0957193,Acting,Hanns Zischler,"Nuremberg, Bavaria, Germany",0.2732,/5O4Dum1OuKI4RMB24xjACkL22iA.jpg
3,378,{266285},False,"[Джонатан Прайс, 조너선 프라이스, 乔纳森·普雷斯, جاناتان پر...","Jonathan Pryce, CBE (born 1 June 1947) is a We...",1947-06-01,None,2,None,nm0000596,Acting,Jonathan Pryce,"Carmel, Flintshire, Wales, UK",2.3491,/zwSv5uXzPTtmitFe39UdqnVwmdL.jpg
4,380,{214137},False,"[Robert DeNiro, Robert Anthony De Niro Jr. , ...","Robert Anthony De Niro (born August 17, 1943) ...",1943-08-17,None,2,None,nm0000134,Acting,Robert De Niro,"Greenwich Village, New York City, New York, USA",4.0718,/cT8htcckIuyI1Lqwt1CvD02ynTh.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15632,5712354,{750223},False,[],,None,None,0,None,None,Acting,Anna Venge Winther,None,0.0000,None
15633,5712356,{750223},False,[],,None,None,0,None,None,Acting,Linette Lindhardt,None,0.0071,None
15634,5712358,{750223},False,[],,None,None,0,None,None,Acting,Signe Louise Pedersen,None,0.0000,None
15635,5712359,{750223},False,[],,None,None,0,None,None,Acting,Mollie Etlar,None,0.0071,None


In [10]:
danish_actors.to_csv("../movie_data/danish_actors.csv", index=False)